In [2]:
%reload_ext autoreload
%autoreload 2
import torch
import dagrad.flex as flex
from dagrad.utils import utils
import numpy as np
from dagrad.flex.prune import cam_pruning

utils.set_random_seed(1)
torch.manual_seed(1)

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
def postprocess(B, graph_thres=0.3):
    """Post-process estimated solution:
        (1) Thresholding.
        (2) Remove the edges with smallest absolute weight until a DAG
            is obtained.

    Args:
        B (numpy.ndarray): [d, d] weighted matrix.
        graph_thres (float): Threshold for weighted matrix. Default: 0.3.

    Returns:
        numpy.ndarray: [d, d] weighted matrix of DAG.
    """
    B = np.copy(B)
    B[np.abs(B) <= graph_thres] = 0    # Thresholding
    B, _ = utils.threshold_till_dag(B)

    return B

## Struct Learn via Flex

`flex.struct_learn` is a function that requires the following 5 inputs:
- `dataset`: A data matrix with `n x d` shape, where `n` is the number of samples and `d` is the number of variables/features.

- `model`: The SEM module. We provide standard SEM implementations such as---LinearModel, LogisticModel, MLP

- `constrained_solver`: An instance of a ConstrainedSolver class. We provide implementations such as `PathFollowing` and `AugmentedLagrangian`

- `unconstrained_solver`: An instance of an UnconstrainedSolver class. We provide implementation for `GradientBasedSolver`

- `loss_fn`: Instance of a Loss class. All available losses can be found at flex/loss.py

- `dag_fn`: Instance of a DagFn class. All available DAG functions can be found at flex/dags.py

## Generate Data

In [3]:
# Generate linear data
n, d, s0, graph_type, sem_type = 1000, 20, 20, "ER", "gauss"
linear_B_true = utils.simulate_dag(d, s0, graph_type)
linear_dataset = utils.simulate_linear_sem(linear_B_true, n, sem_type)

# Generate non-linear data
n, d, s0, graph_type, sem_type = 1000, 5, 5, "ER", "mlp"
nonlinear_B_true = utils.simulate_dag(d, s0, graph_type)
nonlinear_dataset = utils.simulate_nonlinear_sem(nonlinear_B_true, n, sem_type)

# Using flex to implement NOTEARS

## linear NOTEARS

In [12]:
# Linear model
model = flex.LinearModel(d)

# Use AML to solve the constrained problem
cons_solver = flex.AugmentedLagrangian(
    num_iter=10,
    num_steps=[3e4,6e4],
    l1_coeff=0.03,
)

# Use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    optimizer=torch.optim.Adam(model.parameters(), lr=3e-4),
)

# Use MSE loss
loss_fn = flex.MSELoss()

# Use Trace of matrix exponential as DAG function
dag_fn = flex.Exp()

# Learn the DAG
W_est = flex.struct_learn(
    dataset=linear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

acc = utils.count_accuracy(linear_B_true, W_est != 0)
print("Results: ", acc)

100%|██████████| 10/10 [00:37<00:00,  3.79s/it]

Total Time: 37.8957781791687
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 20}


## nonlinear NOTEARS

In [14]:
def grandag_aug_lagrangian(n, d, s0, num_layers=2, noise_type="gauss"):
    graph_type, sem_type = "ER", "mlp"
    nonlinear_B_true = utils.simulate_dag(d, s0, graph_type)
    nonlinear_dataset = utils.simulate_nonlinear_sem(nonlinear_B_true, n, sem_type, noise_type=noise_type)
    # print(f'b_true is {nonlinear_B_true}')

    train_samples = int(nonlinear_dataset.shape[0] * 0.8)
    test_samples = nonlinear_dataset.shape[0] - train_samples
    train_dataset = nonlinear_dataset[:train_samples, :]
    test_dataset = nonlinear_dataset[train_samples:, :]

    # Nonlinear model
    model = flex.MLP(dims=[d, 2, d], num_layers=num_layers, hid_dim=10, activation="sigmoid", bias=True)

    # Use AML to solve the constrained problem
    cons_solver = flex.AugmentedLagrangian(
        num_iter=100000,
        num_steps=[1,1],
        # l1_coeff=0.01,
        # weight_decay=0.01,
        rho_init=1e-3,
    )

    # Use Adam to solve the unconstrained problem
    uncons_solver = flex.GradientBasedSolver(
        optimizer=torch.optim.RMSprop(model.parameters(), lr=1e-3),
    )

    # Use MSE loss
    loss_fn = flex.MSELoss()

    # Use Trace of matrix exponential as DAG function
    dag_fn = flex.Grandag_h()

    # Learn the DAG
    W_est = flex.struct_learn(
        dataset=train_dataset,
        model=model,
        constrained_solver=cons_solver,
        unconstrained_solver=uncons_solver,
        loss_fn=loss_fn,
        dag_fn=dag_fn,
        w_threshold=0.0,
    )

    W_est = postprocess(W_est)

    acc_before = utils.count_accuracy(nonlinear_B_true, W_est != 0)
    print(f'Results before CAM pruning: {acc_before}')

    to_keep = (torch.from_numpy(W_est) > 0).type(torch.Tensor)
    B_est = model.adjacency * to_keep
    # print(f'B_est is {B_est}')

    opt = {
        'cam_pruning_cutoff': np.logspace(-6, 0, 10),
        'exp_path': 'cam_pruning',
    }
    try:
        cam_pruning_cutoff = [float(i) for i in opt['cam_pruning_cutoff']]
    except:
        cam_pruning_cutoff = [float(opt['cam_pruning_cutoff'])]
    for cutoff in cam_pruning_cutoff:
        B_est = cam_pruning(B_est, train_dataset, test_dataset, opt, cutoff=cutoff)
        # print(f'now B_est is {B_est}')
    acc = utils.count_accuracy(nonlinear_B_true, B_est.detach().cpu().numpy() != 0)
    print("Results: ", acc)
    return acc, acc_before

In [72]:
shds = []
for i in range(10):
    acc = grandag_aug_lagrangian(1000, 5, 5, num_layers=2)
    shds.append(acc['shd'])
print(f'mean shd: {np.mean(shds)}')

Using the first value from num_steps for the first 99998 iterations
100%|██████████| 100000/100000 [00:45<00:00, 2207.26it/s]


Total Time: 45.3078351020813
Results before CAM pruning: {'fdr': 0.42857142857142855, 'tpr': 0.8, 'fpr': 0.6, 'shd': 3, 'nnz': 7}


Using the first value from num_steps for the first 99998 iterations


Results:  {'fdr': 0.3333333333333333, 'tpr': 0.8, 'fpr': 0.4, 'shd': 2, 'nnz': 6}


100%|██████████| 100000/100000 [00:28<00:00, 3476.31it/s] 


...

Total Time: 18.262706756591797
Results before CAM pruning: {'fdr': 0.6, 'tpr': 0.8, 'fpr': 1.2, 'shd': 6, 'nnz': 10}


Using the first value from num_steps for the first 99998 iterations


Results:  {'fdr': 0.0, 'tpr': 0.4, 'fpr': 0.0, 'shd': 3, 'nnz': 2}


100%|██████████| 100000/100000 [00:32<00:00, 3080.43it/s]


Total Time: 32.470055103302
Results before CAM pruning: {'fdr': 0.5, 'tpr': 1.0, 'fpr': 1.0, 'shd': 5, 'nnz': 10}
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 5}
mean shd: 2.0


In [7]:
shds = []
for i in range(10):
    acc = grandag_aug_lagrangian(1000, 5, 10, num_layers=2)
    shds.append(acc['shd'])
print(f'mean shd: {np.mean(shds)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 27.739933967590332
Results before CAM pruning: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'sid': 0.0, 'nnz': 10}
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'sid': 0.0, 'nnz': 10}


Using the first value from num_steps for the first 99998 iterations


Total Time: 28.82768940925598
Results before CAM pruning: {'fdr': 0.1111111111111111, 'tpr': 0.8, 'fpr': 1.0, 'shd': 2, 'sid': 11.0, 'nnz': 9}
Results:  {'fdr': 0.25, 'tpr': 0.3, 'fpr': 1.0, 'shd': 7, 'sid': 17.0, 'nnz': 4}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 37.63465762138367
Results before CAM pruning: {'fdr': 0.1111111111111111, 'tpr': 0.8, 'fpr': 1.0, 'shd': 2, 'sid': 5.0, 'nnz': 9}
Results:  {'fdr': 0.1111111111111111, 'tpr': 0.8, 'fpr': 1.0, 'shd': 2, 'sid': 5.0, 'nnz': 9}


Using the first value from num_steps for the first 99998 iterations


Total Time: 33.43969488143921
Results before CAM pruning: {'fdr': 0.5, 'tpr': 0.4, 'fpr': 4.0, 'shd': 6, 'sid': 16.0, 'nnz': 8}
Results:  {'fdr': 0.4, 'tpr': 0.3, 'fpr': 2.0, 'shd': 7, 'sid': 17.0, 'nnz': 5}


Using the first value from num_steps for the first 99998 iterations


Total Time: 36.471829891204834
Results before CAM pruning: {'fdr': 0.1, 'tpr': 0.9, 'fpr': 1.0, 'shd': 1, 'sid': 6.0, 'nnz': 10}
Results:  {'fdr': 0.1, 'tpr': 0.9, 'fpr': 1.0, 'shd': 1, 'sid': 6.0, 'nnz': 10}
mean shd: 4.4


In [8]:
shds = []
for i in range(10):
    acc = grandag_aug_lagrangian(1000, 10, 20, num_layers=2, noise_type="gauss")
    shds.append(acc['shd'])
print(f'mean shd: {np.mean(shds)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 225.47241616249084
Results before CAM pruning: {'fdr': 0.525, 'tpr': 0.95, 'fpr': 0.84, 'shd': 22, 'sid': 2.0, 'nnz': 40}
Results:  {'fdr': 0.10526315789473684, 'tpr': 0.85, 'fpr': 0.08, 'shd': 5, 'sid': 16.0, 'nnz': 19}


Using the first value from num_steps for the first 99998 iterations


Total Time: 200.37697792053223
Results before CAM pruning: {'fdr': 0.5384615384615384, 'tpr': 0.9, 'fpr': 0.84, 'shd': 22, 'sid': 18.0, 'nnz': 39}
Results:  {'fdr': 0.2, 'tpr': 0.8, 'fpr': 0.16, 'shd': 7, 'sid': 24.0, 'nnz': 20}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 370.53898429870605
Results before CAM pruning: {'fdr': 0.45714285714285713, 'tpr': 0.95, 'fpr': 0.64, 'shd': 17, 'sid': 1.0, 'nnz': 35}
Results:  {'fdr': 0.06666666666666667, 'tpr': 0.7, 'fpr': 0.04, 'shd': 7, 'sid': 27.0, 'nnz': 15}


Using the first value from num_steps for the first 99998 iterations


Total Time: 200.92305779457092
Results before CAM pruning: {'fdr': 0.55, 'tpr': 0.9, 'fpr': 0.88, 'shd': 22, 'sid': 14.0, 'nnz': 40}
Results:  {'fdr': 0.17647058823529413, 'tpr': 0.7, 'fpr': 0.12, 'shd': 7, 'sid': 31.0, 'nnz': 17}


Using the first value from num_steps for the first 99998 iterations


Total Time: 253.19810009002686
Results before CAM pruning: {'fdr': 0.5641025641025641, 'tpr': 0.85, 'fpr': 0.88, 'shd': 22, 'sid': 23.0, 'nnz': 39}
Results:  {'fdr': 0.4, 'tpr': 0.45, 'fpr': 0.24, 'shd': 14, 'sid': 43.0, 'nnz': 15}
mean shd: 9.0


In [9]:
shds = []
for i in range(10):
    acc = grandag_aug_lagrangian(1000, 20, 40, num_layers=2, noise_type="gauss")
    shds.append(acc['shd'])
print(f'mean shd: {np.mean(shds)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 404.33473110198975
Results before CAM pruning: {'fdr': 0.782608695652174, 'tpr': 0.875, 'fpr': 0.84, 'shd': 128, 'sid': 65.0, 'nnz': 161}
Results:  {'fdr': 0.20588235294117646, 'tpr': 0.675, 'fpr': 0.04666666666666667, 'shd': 17, 'sid': 131.0, 'nnz': 34}


Using the first value from num_steps for the first 99998 iterations


Total Time: 296.53825187683105
Results before CAM pruning: {'fdr': 0.7625, 'tpr': 0.95, 'fpr': 0.8133333333333334, 'shd': 123, 'sid': 22.0, 'nnz': 160}
Results:  {'fdr': 0.06896551724137931, 'tpr': 0.675, 'fpr': 0.013333333333333334, 'shd': 14, 'sid': 109.0, 'nnz': 29}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 288.392498254776
Results before CAM pruning: {'fdr': 0.8206896551724138, 'tpr': 0.65, 'fpr': 0.7933333333333333, 'shd': 128, 'sid': 140.0, 'nnz': 145}
Results:  {'fdr': 0.3181818181818182, 'tpr': 0.375, 'fpr': 0.04666666666666667, 'shd': 32, 'sid': 155.0, 'nnz': 22}


Using the first value from num_steps for the first 99998 iterations


Total Time: 346.60823917388916
Results before CAM pruning: {'fdr': 0.8108108108108109, 'tpr': 0.525, 'fpr': 0.6, 'shd': 102, 'sid': 192.0, 'nnz': 111}
Results:  {'fdr': 0.3076923076923077, 'tpr': 0.45, 'fpr': 0.05333333333333334, 'shd': 28, 'sid': 141.0, 'nnz': 26}


Using the first value from num_steps for the first 99998 iterations


Total Time: 338.1101818084717
Results before CAM pruning: {'fdr': 0.8133333333333334, 'tpr': 0.7, 'fpr': 0.8133333333333334, 'shd': 127, 'sid': 129.0, 'nnz': 150}
Results:  {'fdr': 0.2222222222222222, 'tpr': 0.525, 'fpr': 0.04, 'shd': 22, 'sid': 153.0, 'nnz': 27}
mean shd: 19.6


In [11]:
shds = []
for i in range(10):
    acc = grandag_aug_lagrangian(1000, 5, 10, num_layers=2, noise_type="exp")
    shds.append(acc['shd'])
print(f'mean shd: {np.mean(shds)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 52.67776107788086
Results before CAM pruning: {'fdr': 0.4444444444444444, 'tpr': 0.5, 'fpr': 4.0, 'shd': 5, 'sid': 15.0, 'nnz': 9}
Results:  {'fdr': 0.5, 'tpr': 0.4, 'fpr': 4.0, 'shd': 6, 'sid': 16.0, 'nnz': 8}


Using the first value from num_steps for the first 99998 iterations


Total Time: 55.82098722457886
Results before CAM pruning: {'fdr': 0.0, 'tpr': 0.8, 'fpr': 0.0, 'shd': 2, 'sid': 5.0, 'nnz': 8}
Results:  {'fdr': 0.0, 'tpr': 0.8, 'fpr': 0.0, 'shd': 2, 'sid': 5.0, 'nnz': 8}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 42.479065895080566
Results before CAM pruning: {'fdr': 0.4, 'tpr': 0.6, 'fpr': 4.0, 'shd': 4, 'sid': 14.0, 'nnz': 10}
Results:  {'fdr': 0.4, 'tpr': 0.6, 'fpr': 4.0, 'shd': 4, 'sid': 14.0, 'nnz': 10}


Using the first value from num_steps for the first 99998 iterations


Total Time: 37.52192711830139
Results before CAM pruning: {'fdr': 0.8, 'tpr': 0.2, 'fpr': 8.0, 'shd': 8, 'sid': 18.0, 'nnz': 10}
Results:  {'fdr': 0.8571428571428571, 'tpr': 0.1, 'fpr': 6.0, 'shd': 9, 'sid': 19.0, 'nnz': 7}


Using the first value from num_steps for the first 99998 iterations


Total Time: 28.148314952850342
Results before CAM pruning: {'fdr': 0.7, 'tpr': 0.3, 'fpr': 7.0, 'shd': 7, 'sid': 17.0, 'nnz': 10}
Results:  {'fdr': 0.8333333333333334, 'tpr': 0.1, 'fpr': 5.0, 'shd': 9, 'sid': 16.0, 'nnz': 6}
mean shd: 5.4


In [12]:
shds = []
for i in range(10):
    acc = grandag_aug_lagrangian(1000, 10, 20, num_layers=2, noise_type="exp")
    shds.append(acc['shd'])
print(f'mean shd: {np.mean(shds)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 198.1605167388916
Results before CAM pruning: {'fdr': 0.813953488372093, 'tpr': 0.4, 'fpr': 1.4, 'shd': 37, 'sid': 51.0, 'nnz': 43}
Results:  {'fdr': 0.6842105263157895, 'tpr': 0.3, 'fpr': 0.52, 'shd': 21, 'sid': 65.0, 'nnz': 19}


Using the first value from num_steps for the first 99998 iterations


Total Time: 227.23774003982544
Results before CAM pruning: {'fdr': 0.7105263157894737, 'tpr': 0.55, 'fpr': 1.08, 'shd': 29, 'sid': 47.0, 'nnz': 38}
Results:  {'fdr': 0.7142857142857143, 'tpr': 0.2, 'fpr': 0.4, 'shd': 20, 'sid': 66.0, 'nnz': 14}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 224.1187081336975
Results before CAM pruning: {'fdr': 0.5952380952380952, 'tpr': 0.85, 'fpr': 1.0, 'shd': 25, 'sid': 25.0, 'nnz': 42}
Results:  {'fdr': 0.26666666666666666, 'tpr': 0.55, 'fpr': 0.16, 'shd': 10, 'sid': 46.0, 'nnz': 15}


Using the first value from num_steps for the first 99998 iterations


Total Time: 250.54882788658142
Results before CAM pruning: {'fdr': 0.7428571428571429, 'tpr': 0.45, 'fpr': 1.04, 'shd': 27, 'sid': 56.0, 'nnz': 35}
Results:  {'fdr': 0.5555555555555556, 'tpr': 0.4, 'fpr': 0.4, 'shd': 14, 'sid': 68.0, 'nnz': 18}


Using the first value from num_steps for the first 99998 iterations


Total Time: 233.3630073070526
Results before CAM pruning: {'fdr': 0.7647058823529411, 'tpr': 0.4, 'fpr': 1.04, 'shd': 28, 'sid': 69.0, 'nnz': 34}
Results:  {'fdr': 0.6842105263157895, 'tpr': 0.3, 'fpr': 0.52, 'shd': 19, 'sid': 77.0, 'nnz': 19}
mean shd: 15.8


In [13]:
shds = []
for i in range(10):
    acc = grandag_aug_lagrangian(1000, 20, 40, num_layers=2, noise_type="exp")
    shds.append(acc['shd'])
print(f'mean shd: {np.mean(shds)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 9702.126415967941
Results before CAM pruning: {'fdr': 0.8255813953488372, 'tpr': 0.75, 'fpr': 0.9466666666666667, 'shd': 148, 'sid': 94.0, 'nnz': 172}
Results:  {'fdr': 0.2903225806451613, 'tpr': 0.55, 'fpr': 0.06, 'shd': 24, 'sid': 159.0, 'nnz': 31}


Using the first value from num_steps for the first 99998 iterations


Total Time: 402.982216835022
Results before CAM pruning: {'fdr': 0.8343949044585988, 'tpr': 0.65, 'fpr': 0.8733333333333333, 'shd': 133, 'sid': 113.0, 'nnz': 157}
Results:  {'fdr': 0.48484848484848486, 'tpr': 0.425, 'fpr': 0.10666666666666667, 'shd': 29, 'sid': 194.0, 'nnz': 33}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 431.742063999176
Results before CAM pruning: {'fdr': 0.8128654970760234, 'tpr': 0.8, 'fpr': 0.9266666666666666, 'shd': 139, 'sid': 76.0, 'nnz': 171}
Results:  {'fdr': 0.42857142857142855, 'tpr': 0.4, 'fpr': 0.08, 'shd': 28, 'sid': 206.0, 'nnz': 28}


Using the first value from num_steps for the first 99998 iterations


Total Time: 340.2285740375519
Results before CAM pruning: {'fdr': 0.8323699421965318, 'tpr': 0.725, 'fpr': 0.96, 'shd': 145, 'sid': 87.0, 'nnz': 173}
Results:  {'fdr': 0.375, 'tpr': 0.5, 'fpr': 0.08, 'shd': 25, 'sid': 218.0, 'nnz': 32}


Using the first value from num_steps for the first 99998 iterations


Total Time: 388.3434510231018
Results before CAM pruning: {'fdr': 0.8294117647058824, 'tpr': 0.725, 'fpr': 0.94, 'shd': 143, 'sid': 159.0, 'nnz': 170}
Results:  {'fdr': 0.34210526315789475, 'tpr': 0.625, 'fpr': 0.08666666666666667, 'shd': 21, 'sid': 153.0, 'nnz': 38}
mean shd: 25.5


## NO CAM

In [16]:
shds = []
sids = []
for i in range(10):
    _, acc_before = grandag_aug_lagrangian(1000, 5, 10, num_layers=2, noise_type="exp")
    shds.append(acc_before['shd'])
    sids.append(acc_before['sid'])
print(f'mean shd: {np.mean(shds)}')
print(f'mean sid: {np.mean(sids)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 51.29020094871521
Results before CAM pruning: {'fdr': 0.2222222222222222, 'tpr': 0.7, 'fpr': 2.0, 'shd': 3, 'sid': 13.0, 'nnz': 9}
Results:  {'fdr': 0.25, 'tpr': 0.6, 'fpr': 2.0, 'shd': 4, 'sid': 14.0, 'nnz': 8}


Using the first value from num_steps for the first 99998 iterations


Total Time: 35.35465908050537
Results before CAM pruning: {'fdr': 0.5555555555555556, 'tpr': 0.4, 'fpr': 5.0, 'shd': 6, 'sid': 16.0, 'nnz': 9}
Results:  {'fdr': 0.5555555555555556, 'tpr': 0.4, 'fpr': 5.0, 'shd': 6, 'sid': 16.0, 'nnz': 9}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 41.29415416717529
Results before CAM pruning: {'fdr': 0.7777777777777778, 'tpr': 0.2, 'fpr': 7.0, 'shd': 8, 'sid': 18.0, 'nnz': 9}
Results:  {'fdr': 0.7777777777777778, 'tpr': 0.2, 'fpr': 7.0, 'shd': 8, 'sid': 18.0, 'nnz': 9}


Using the first value from num_steps for the first 99998 iterations


Total Time: 42.10661578178406
Results before CAM pruning: {'fdr': 0.5, 'tpr': 0.5, 'fpr': 5.0, 'shd': 5, 'sid': 15.0, 'nnz': 10}
Results:  {'fdr': 0.4444444444444444, 'tpr': 0.5, 'fpr': 4.0, 'shd': 5, 'sid': 15.0, 'nnz': 9}


Using the first value from num_steps for the first 99998 iterations


Total Time: 46.78558588027954
Results before CAM pruning: {'fdr': 0.375, 'tpr': 0.5, 'fpr': 3.0, 'shd': 5, 'sid': 8.0, 'nnz': 8}
Results:  {'fdr': 0.375, 'tpr': 0.5, 'fpr': 3.0, 'shd': 5, 'sid': 8.0, 'nnz': 8}
mean shd: 4.9
mean sid: 12.9


In [17]:
shds = []
sids = []
for i in range(10):
    _, acc_before = grandag_aug_lagrangian(1000, 5, 10, num_layers=2, noise_type="gauss")
    shds.append(acc_before['shd'])
    sids.append(acc_before['sid'])
print(f'mean shd: {np.mean(shds)}')
print(f'mean sid: {np.mean(sids)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 39.443745136260986
Results before CAM pruning: {'fdr': 0.0, 'tpr': 0.9, 'fpr': 0.0, 'shd': 1, 'sid': 2.0, 'nnz': 9}
Results:  {'fdr': 0.0, 'tpr': 0.9, 'fpr': 0.0, 'shd': 1, 'sid': 2.0, 'nnz': 9}


Using the first value from num_steps for the first 99998 iterations


Total Time: 43.48696422576904
Results before CAM pruning: {'fdr': 0.0, 'tpr': 0.9, 'fpr': 0.0, 'shd': 1, 'sid': 3.0, 'nnz': 9}
Results:  {'fdr': 0.0, 'tpr': 0.7, 'fpr': 0.0, 'shd': 3, 'sid': 6.0, 'nnz': 7}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 786.8809661865234
Results before CAM pruning: {'fdr': 0.2, 'tpr': 0.8, 'fpr': 2.0, 'shd': 2, 'sid': 8.0, 'nnz': 10}
Results:  {'fdr': 0.2222222222222222, 'tpr': 0.7, 'fpr': 2.0, 'shd': 3, 'sid': 9.0, 'nnz': 9}


Using the first value from num_steps for the first 99998 iterations


Total Time: 270.13709902763367
Results before CAM pruning: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'sid': 0.0, 'nnz': 10}
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'sid': 0.0, 'nnz': 10}


Using the first value from num_steps for the first 99998 iterations


Total Time: 5311.203251123428
Results before CAM pruning: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'sid': 0.0, 'nnz': 10}
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'sid': 0.0, 'nnz': 10}
mean shd: 1.5
mean sid: 4.1


In [18]:
shds = []
sids = []
for i in range(10):
    _, acc_before = grandag_aug_lagrangian(1000, 5, 10, num_layers=2, noise_type="gumbel")
    shds.append(acc_before['shd'])
    sids.append(acc_before['sid'])
print(f'mean shd: {np.mean(shds)}')
print(f'mean sid: {np.mean(sids)}')

Using the first value from num_steps for the first 99998 iterations


Total Time: 5628.243134975433
Results before CAM pruning: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'sid': 0.0, 'nnz': 10}
Results:  {'fdr': 0.0, 'tpr': 0.7, 'fpr': 0.0, 'shd': 3, 'sid': 6.0, 'nnz': 7}


Using the first value from num_steps for the first 99998 iterations


Total Time: 50.781733751297
Results before CAM pruning: {'fdr': 0.1111111111111111, 'tpr': 0.8, 'fpr': 1.0, 'shd': 2, 'sid': 5.0, 'nnz': 9}
Results:  {'fdr': 0.16666666666666666, 'tpr': 0.5, 'fpr': 1.0, 'shd': 5, 'sid': 8.0, 'nnz': 6}


Using the first value from num_steps for the first 99998 iterations


...

Total Time: 34.75255298614502
Results before CAM pruning: {'fdr': 0.6, 'tpr': 0.4, 'fpr': 6.0, 'shd': 6, 'sid': 16.0, 'nnz': 10}
Results:  {'fdr': 0.6, 'tpr': 0.4, 'fpr': 6.0, 'shd': 6, 'sid': 16.0, 'nnz': 10}


Using the first value from num_steps for the first 99998 iterations


Total Time: 30.556212902069092
Results before CAM pruning: {'fdr': 0.5, 'tpr': 0.5, 'fpr': 5.0, 'shd': 5, 'sid': 15.0, 'nnz': 10}
Results:  {'fdr': 0.6666666666666666, 'tpr': 0.2, 'fpr': 4.0, 'shd': 8, 'sid': 18.0, 'nnz': 6}


Using the first value from num_steps for the first 99998 iterations


Total Time: 49.62290596961975
Results before CAM pruning: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'sid': 0.0, 'nnz': 10}
Results:  {'fdr': 0.0, 'tpr': 0.7, 'fpr': 0.0, 'shd': 3, 'sid': 6.0, 'nnz': 7}
mean shd: 3.0
mean sid: 8.3


In [77]:
grandag_aug_lagrangian(1000, 5, 5, num_layers=2, noise_type='exp')

Using the first value from num_steps for the first 99998 iterations
100%|██████████| 100000/100000 [11:11<00:00, 148.89it/s] 


Total Time: 671.6974756717682
Results before CAM pruning: {'fdr': 0.8, 'tpr': 0.4, 'fpr': 1.6, 'shd': 8, 'sid': 15.0, 'nnz': 10}
Results:  {'fdr': 0.6666666666666666, 'tpr': 0.4, 'fpr': 0.8, 'shd': 4, 'sid': 16.0, 'nnz': 6}


{'fdr': 0.6666666666666666,
 'tpr': 0.4,
 'fpr': 0.8,
 'shd': 4,
 'sid': 16.0,
 'nnz': 6}

In [78]:
grandag_aug_lagrangian(1000, 5, 5, num_layers=2, noise_type='gumbel')

Using the first value from num_steps for the first 99998 iterations


Total Time: 976.2653551101685
Results before CAM pruning: {'fdr': 0.5714285714285714, 'tpr': 0.6, 'fpr': 0.8, 'shd': 4, 'sid': 7.0, 'nnz': 7}
Results:  {'fdr': 0.5, 'tpr': 0.4, 'fpr': 0.4, 'shd': 3, 'sid': 12.0, 'nnz': 4}


{'fdr': 0.5, 'tpr': 0.4, 'fpr': 0.4, 'shd': 3, 'sid': 12.0, 'nnz': 4}

In [76]:
grandag_aug_lagrangian(1000, 10, 10, num_layers=2)

100%|██████████| 100000/100000 [04:59<00:00, 334.20it/s]


Total Time: 299.23108100891113
Results before CAM pruning: {'fdr': 0.76, 'tpr': 0.6, 'fpr': 0.5428571428571428, 'shd': 23, 'sid': 13.0, 'nnz': 25}
Results:  {'fdr': 0.4444444444444444, 'tpr': 0.5, 'fpr': 0.11428571428571428, 'shd': 9, 'sid': 20.0, 'nnz': 9}


{'fdr': 0.4444444444444444,
 'tpr': 0.5,
 'fpr': 0.11428571428571428,
 'shd': 9,
 'sid': 20.0,
 'nnz': 9}

In [37]:
grandag_aug_lagrangian(1000, 20, 20, num_layers=2)

/Users/seren/dagrad/dagrad/flex/modules/constrained_solvers.py:160: UserWarning: Using the first value from num_steps for the first 99998 iterations
  self.vwarn(f"Using the first value from num_steps for the first {missing_steps} iterations")


b_true is [[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

100%|██████████| 100000/100000 [07:06<00:00, 234.64it/s] 

h new is 0.0 and h tol is 1e-08 so ending at 13021th iteration
Total Time: 426.18568086624146
Results:  {'fdr': 0.8689655172413793, 'tpr': 0.95, 'fpr': 0.7411764705882353, 'shd': 126, 'nnz': 145}


# Using flex to implement DAGMA

## Linear DAGMA

In [14]:
# Linear model
model = flex.LinearModel(d)

# Use path following to solve the constrained problem
cons_solver = flex.PathFollowing(
    num_iter=5,
    mu_init=1.0,
    mu_scale=0.1,
    logdet_coeff=[1.0, .9, .8, .7, .6],
    num_steps=[3e4, 6e4],
    l1_coeff=0.03,
)

# use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    optimizer=torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.99, 0.999))
)

# Use MSE loss
loss_fn = flex.MSELoss()

# Use LogDet as DAG function
dag_fn = flex.LogDet()

W_est = flex.struct_learn(
    dataset=linear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

acc = utils.count_accuracy(linear_B_true, W_est != 0)
print("Results: ", acc)

/Users/seren/dagrad/dagrad/flex/modules/constrained_solvers.py:49: UserWarning: Using the first value from num_steps for the first 3 iterations
  self.vwarn(
100%|██████████| 5/5 [00:08<00:00,  1.77s/it]

Total Time: 8.867684841156006
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 20}


## Nonlinear DAGMA

In [48]:
n, d, s0, graph_type, sem_type = 1000, 5, 5, "ER", "mlp"
nonlinear_B_true = utils.simulate_dag(d, s0, graph_type)
nonlinear_dataset = utils.simulate_nonlinear_sem(nonlinear_B_true, n, sem_type)

# nonlinear model
model = flex.MLP(dims=[d, 2, 5], num_layers=2, hid_dim=10, activation="sigmoid", bias=True)

# Use path following to solve the constrained problem
cons_solver = flex.PathFollowing(
    num_iter=4,
    mu_init=0.1,
    mu_scale=0.1,
    logdet_coeff=1.0,
    num_steps=[5e4, 8e4],
    weight_decay=0.02,
    l1_coeff=0.005,
)

# use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    # optimizer=torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.99, 0.999))
    optimizer=torch.optim.SGD(model.parameters(), lr=2e-4)
)

# Use NLL loss
loss_fn = flex.NLLLoss()

# Use LogDet as DAG function
dag_fn = flex.LogDet()

# Learn the DAG
W_est = flex.struct_learn(
    dataset=nonlinear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

W_est = postprocess(W_est)
acc = utils.count_accuracy(nonlinear_B_true, W_est != 0)
print("Results: ", acc)

/Users/seren/dagrad/dagrad/flex/modules/constrained_solvers.py:51: UserWarning: Using the first value from num_steps for the first 2 iterations
  self.vwarn(
100%|██████████| 4/4 [09:40<00:00, 145.12s/it]

Total Time: 580.489175081253
Results:  {'fdr': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 5, 'nnz': 0}
